In [1]:
import numpy as np
import pandas as pd
from siuba import *
from calitp import *
import intake
import shared_utils

import data_prep
import _utils

/opt/conda/lib/python3.9/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(
/opt/conda/lib/python3.9/site-packages/google/auth/_default.py:79: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)
E0321 22:07:57.740766942    1265 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies
/opt/conda/lib/python3.9/site-packages/google/auth/_defau

In [2]:
import altair as alt
import altair_saver
from shared_utils import geography_utils
from shared_utils import altair_utils
from shared_utils import calitp_color_palette as cp
from shared_utils import styleguide

In [3]:
pd.set_option("display.max_columns", 100)

### Read in BC_GTFS_NTD_2

In [4]:
df= pd.read_parquet('BC_GTFS_NTD_2.parquet')

In [5]:
df.sample(5)

,grant_fiscal_year,funding_program,grant_number,project_year,organization_name,upin,description,ali,contract_number,allocationamount,encumbered_amount,expendedamount,activebalance,closedoutbalance,project_status,ntd_id,reporter_type,_0_9,_10_12,_13_15,_16_20,_21_25,_26_30,_31_60,_60plus,total_vehicles,average_age_of_fleet__in_years_,average_lifetime_miles_per_vehicle,sum_15plus,Automobiles,Bus,Other,Service,Train,Van,automobiles_door,bus_doors,train_doors,van_doors,doors_sum,itp_id,Is_Agency_In_BC_Only_1_means_Yes,GTFS,fleet_size,adjusted_allocationamount,adjusted_encumbered_amount,adjusted_expendedamount,adjusted_activebalance,adjusted_closedoutbalance,_16plus,name,caltrans_district,mpo_rtpa,planning_authority,gtfs_static_status,gtfs_realtime_status,year
23,2016,Section 5311,CA-2017-025 | 0017000158,2016,City of Auburn,BCG0000137,Operating Assistance,300902,64BO17-00366,68989.0,68989.0,68989.0,0.0,0,Open,9R02-91032,Rural Reporter,5.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,6.0,6.250000,0.000000,NaN,0.0,2.0,0.0,1.0,0.0,3.0,0.0,4.0,0.0,3.0,7.0,23.0,0,Ok,Small,77889.133846,77889.133846,77889.133846,0.000000,0.0,1.0,City of Auburn,03 - Marysville,Sacramento Area Council of Governments,Placer County Transportation Planning Agency,Static OK,RT Incomplete,2016-01-01
105,2017,Section 5311,CA-2017-168 | 0017000259,2017,City of Auburn,BCG0001264,Operating Assistance Sliding Scale,300902,64BO17-00512,78928.0,78928.0,78928.0,0.0,0,Open,9R02-91032,Rural Reporter,5.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,6.0,6.250000,0.000000,NaN,0.0,2.0,0.0,1.0,0.0,3.0,0.0,4.0,0.0,3.0,7.0,23.0,0,Ok,Small,87251.769412,87251.769412,87251.769412,0.000000,0.0,1.0,City of Auburn,03 - Marysville,Sacramento Area Council of Governments,Placer County Transportation Planning Agency,Static OK,RT Incomplete,2017-01-01
778,2021,Section 5311,TBD | 0021000227,2021,San Luis Obispo Regional Transit Authority,BCG0002844,Operating Assistance Sliding Scale,300902,None,586252.0,0.0,0.0,586252.0,0,Open,90206,Full Reporter,76.0,5.0,0.0,5.0,0.0,0.0,0.0,0.0,86.0,8.416162,204346.553050,NaN,0.0,36.0,0.0,17.0,0.0,33.0,0.0,72.0,0.0,33.0,105.0,289.0,0,Ok,Large,586252.499277,0.000000,0.000000,586252.499277,0.0,5.0,None,None,None,None,None,None,2021-01-01
341,2019,Section 5311(f),CA-2018-114 | 0019000012,2019,Monterey-Salinas Transit,BCG0001434,FY19 Line 85 Operating Assistance Sliding Scale,300902,64CO19-00901,283186.0,283186.0,283186.0,0.0,0,Open,90062,Full Reporter,151.0,15.0,14.0,16.0,0.0,0.0,0.0,0.0,196.0,7.128430,317142.100475,NaN,0.0,93.0,0.0,38.0,0.0,65.0,0.0,186.0,0.0,65.0,251.0,208.0,0,Research,Large,300147.443562,300147.443562,300147.443562,0.000000,0.0,16.0,Monterey-Salinas Transit,05 - San Luis Obispo,Association of Monterey Bay Area Governments,Transportation Agency for Monterey County,Static Incomplete,RT OK,2019-01-01
300,2019,Section 5311,CA-2020-015 | 0020000057,2019,Kings County Area Public Transit Agency,BCG0001539,Operating Assistance Sliding Scale,300902,64BO19-00957,105192.0,292918.0,187726.0,-82534.0,0,Open,90200,Full Reporter,27.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,34.0,6.092593,105613.885533,NaN,0.0,18.0,0.0,3.0,0.0,13.0,0.0,36.0,0.0,13.0,49.0,148.0,0,Ok,Medium,111492.481560,310462.342324,198969.860763,-87477.379203,0.0,0.0,Kings County Area Public Transit Agency,06 - Fresno,Kings County Association of Governments,Kings County Association of Governments,Static OK,RT OK,2019-01-01


In [6]:
df>>count(_.caltrans_district)

,caltrans_district,n
0,01 - Eureka,75
1,02 - Redding,29
2,03 - Marysville,71
3,04 - Oakland,49
4,05 - San Luis Obispo,56
5,06 - Fresno,107
6,07 - Los Angeles,10
7,08 - San Bernardino,78
8,09 - Bishop,36
9,10 - Stockton,31


### Add Airtable 

In [17]:
# GCS_FILE_PATH = "gs://calitp-analytics-data/data-analyses/5311 /"
# File_Organization =  "organizations-AllOrganizations_1.csv"
# organizations =  pd.read_csv(f'{GCS_FILE_PATH}{File_Organization}')
# airtable = to_snakecase(organizations)

# airtable = airtable>>select(_.name,
#                       # _.ntp_id,
#                        # _.itp_id,
#                       _.caltrans_district,
#                       _.mpo_rtpa,
#                       _.planning_authority,
#                       _.gtfs_static_status,
#                       _.gtfs_realtime_status
#                      )    

In [18]:
#df = pd.merge(df, airtable, how='left', left_on='organization_name', right_on='name')

In [19]:
#df = df.drop('name', 1)

In [13]:
#df.to_parquet("BC_GTFS_NTD_2.parquet")

### Read in 5311 Agg 

In [10]:
df1 = pd.read_parquet('5311_agg.parquet')

## District 3

In [12]:
df_all = df>>filter(_.caltrans_district=='04 - Oakland')
df_all.head()

,grant_fiscal_year,funding_program,grant_number,project_year,organization_name,upin,description,ali,contract_number,allocationamount,encumbered_amount,expendedamount,activebalance,closedoutbalance,project_status,ntd_id,reporter_type,_0_9,_10_12,_13_15,_16_20,_21_25,_26_30,_31_60,_60plus,total_vehicles,average_age_of_fleet__in_years_,average_lifetime_miles_per_vehicle,sum_15plus,Automobiles,Bus,Other,Service,Train,Van,automobiles_door,bus_doors,train_doors,van_doors,doors_sum,itp_id,Is_Agency_In_BC_Only_1_means_Yes,GTFS,fleet_size,adjusted_allocationamount,adjusted_encumbered_amount,adjusted_expendedamount,adjusted_activebalance,adjusted_closedoutbalance,_16plus,name,caltrans_district,mpo_rtpa,planning_authority,gtfs_static_status,gtfs_realtime_status,year
25,2016,Section 5311,CA-2017-025 | 0017000158,2016,City of Dixon,BCG0000163,Operating Assistance,300902,64BO17-00371,183266.0,183266.0,183266.0,0.0,0,Open,9R02-91041,Rural Reporter,8.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,10.0,7.812500,0.000000,NaN,0.0,0.0,0.0,0.0,0.0,10.0,0.0,0.0,0.0,10.0,10.0,94.0,0,Needed,Medium,206908.782610,206908.782610,206908.782610,0.0,0.0,0.0,City of Dixon,04 - Oakland,Metropolitan Transportation Commission,Metropolitan Transportation Commission,Static OK,RT Incomplete,2016-01-01
26,2016,Section 5311,CA-2017-025 | 0017000158,2016,City of Fairfield,BCG0000231,Operating Assistance,300902,64BO17-00374,100000.0,100000.0,100000.0,0.0,0,Open,90092,Full Reporter,32.0,5.0,9.0,19.0,0.0,0.0,0.0,0.0,65.0,9.843265,328473.306633,NaN,0.0,48.0,0.0,5.0,0.0,12.0,0.0,96.0,0.0,12.0,108.0,110.0,0,Ok,Medium,112900.801354,112900.801354,112900.801354,0.0,0.0,19.0,City of Fairfield,04 - Oakland,Metropolitan Transportation Commission,Metropolitan Transportation Commission,Static OK,RT OK,2016-01-01
33,2016,Section 5311,CA-2017-025 | 0017000158,2016,City of Rio Vista,BCG0000091,Operating Assistance,300902,64BO17-00382,105000.0,105000.0,105000.0,0.0,0,Open,9R02-91014,Rural Reporter,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,6.900000,0.000000,NaN,0.0,0.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0,6.0,6.0,264.0,0,Ok,Small,118545.841422,118545.841422,118545.841422,0.0,0.0,0.0,City of Rio Vista,04 - Oakland,Metropolitan Transportation Commission,Metropolitan Transportation Commission,Static OK,RT Incomplete,2016-01-01
53,2016,Section 5311,CA-2017-025 | 0017000158,2016,Marin County Transit District,BCG0000229,Operating Assistance,300902,64BO17-00410,200063.0,200063.0,200063.0,0.0,0,Open,90234,Full Reporter,106.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,116.0,4.451499,128748.808633,NaN,0.0,56.0,0.0,2.0,0.0,58.0,0.0,112.0,0.0,58.0,170.0,194.0,0,Ok,Large,225872.730213,225872.730213,225872.730213,0.0,0.0,0.0,Marin County Transit District,04 - Oakland,Metropolitan Transportation Commission,Metropolitan Transportation Commission,Static Incomplete,RT OK,2016-01-01
58,2016,Section 5311,CA-2017-025 | 0017000158,2016,Napa Valley Transportation Authority,BCG0000128,Operating Assistance,300902,64BO17-00416,200991.0,200991.0,200991.0,0.0,0,Open,90088,Full Reporter,66.0,3.0,1.0,8.0,0.0,0.0,0.0,0.0,78.0,7.515625,222854.095200,NaN,0.0,44.0,0.0,2.0,0.0,32.0,0.0,88.0,0.0,32.0,120.0,218.0,0,Ok,Medium,226920.449650,226920.449650,226920.449650,0.0,0.0,8.0,Napa Valley Transportation Authority,04 - Oakland,Metropolitan Transportation Commission,Metropolitan Transportation Commission,Static OK,RT Incomplete,2016-01-01


In [13]:
len(df_all)

49

In [29]:
df2 = df1>>filter(_.caltrans_district=='04 - Oakland')>>arrange(-_.adjusted_allocationamount)
df2>>arrange(_.fleet_size)

,organization_name,reporter_type,fleet_size,ntd_id,itp_id,allocationamount,encumbered_amount,expendedamount,activebalance,closedoutbalance,adjusted_allocationamount,adjusted_expendedamount,adjusted_encumbered_amount,adjusted_activebalance,adjusted_closedoutbalance,total_vehicles,average_age_of_fleet__in_years_,average_lifetime_miles_per_vehicle,Automobiles,Bus,Other,Train,Van,automobiles_door,bus_doors,van_doors,train_doors,doors_sum,_0_9,_10_12,_13_15,_16_20,_21_25,_26_30,_31_60,_16plus,_60plus,caltrans_district,mpo_rtpa,planning_authority,gtfs_static_status,gtfs_realtime_status
31,Marin County Transit District,Full Reporter,Large,90234,194.0,214857.666667,176741.833333,176741.833333,38115.833333,0.0,229480.635880,191364.770086,191364.770086,38115.865794,0.0,116.0,4.451499,128748.808633,0.0,56.0,0.0,0.0,58.0,0.0,112.0,58.0,0.0,170.0,106.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,04 - Oakland,Metropolitan Transportation Commission,Metropolitan Transportation Commission,Static Incomplete,RT OK
42,San Mateo County Transit District,Full Reporter,Large,90009,290.0,159212.666667,131286.000000,131286.000000,27926.666667,0.0,170078.971124,142152.280674,142152.280674,27926.690450,0.0,578.0,7.201646,168268.758600,11.0,401.0,0.0,0.0,84.0,22.0,802.0,84.0,0.0,908.0,305.0,171.0,0.0,102.0,0.0,0.0,0.0,102.0,0.0,04 - Oakland,Metropolitan Transportation Commission,Metropolitan Transportation Commission,Static Incomplete,RT Incomplete
43,Santa Clara Valley Transportation Authority,Full Reporter,Large,90013,294.0,81896.166667,67422.833333,67422.833333,14473.333333,0.0,87475.207774,73001.862114,73001.862114,14473.345659,0.0,1114.0,19.841354,217644.641686,76.0,483.0,0.0,99.0,141.0,152.0,966.0,141.0,198.0,1457.0,645.0,126.0,24.0,283.0,27.0,4.0,4.0,318.0,1.0,04 - Oakland,Metropolitan Transportation Commission,Metropolitan Transportation Commission,Static OK,RT OK
36,Napa Valley Transportation Authority,Full Reporter,Medium,90088,218.0,242859.818182,197218.545455,186143.389091,56716.429091,0.0,258514.820234,201135.005075,212873.508636,57379.815159,0.0,78.0,7.515625,222854.095200,0.0,44.0,0.0,0.0,32.0,0.0,88.0,32.0,0.0,120.0,66.0,3.0,1.0,8.0,0.0,0.0,0.0,8.0,0.0,04 - Oakland,Metropolitan Transportation Commission,Metropolitan Transportation Commission,Static OK,RT Incomplete
10,City of Dixon,Rural Reporter,Medium,9R02-91041,94.0,170903.555556,143539.222222,109636.666667,61266.888889,0.0,182445.724137,119586.083225,155081.367499,62859.640911,0.0,10.0,7.812500,0.000000,0.0,0.0,0.0,0.0,10.0,0.0,0.0,10.0,0.0,10.0,8.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,04 - Oakland,Metropolitan Transportation Commission,Metropolitan Transportation Commission,Static OK,RT Incomplete
11,City of Fairfield,Full Reporter,Medium,90092,110.0,100000.000000,100000.000000,100000.000000,0.000000,0.0,111723.414951,111723.414951,111723.414951,0.000000,0.0,65.0,9.843265,328473.306633,0.0,48.0,0.0,0.0,12.0,0.0,96.0,12.0,0.0,108.0,32.0,5.0,9.0,19.0,0.0,0.0,0.0,19.0,0.0,04 - Oakland,Metropolitan Transportation Commission,Metropolitan Transportation Commission,Static OK,RT OK
17,City of Rio Vista,Rural Reporter,Small,9R02-91014,264.0,83945.555556,63166.666667,54833.333333,29112.222222,0.0,88741.502064,59130.136614,67962.595479,29611.365451,0.0,6.0,6.900000,0.000000,0.0,0.0,0.0,0.0,6.0,0.0,0.0,6.0,0.0,6.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,04 - Oakland,Metropolitan Transportation Commission,Metropolitan Transportation Commission,Static OK,RT Incomplete


In [15]:
df2.describe()

,itp_id,allocationamount,encumbered_amount,expendedamount,activebalance,closedoutbalance,adjusted_allocationamount,adjusted_expendedamount,adjusted_encumbered_amount,adjusted_activebalance,adjusted_closedoutbalance,total_vehicles,average_age_of_fleet__in_years_,average_lifetime_miles_per_vehicle,Automobiles,Bus,Other,Train,Van,automobiles_door,bus_doors,van_doors,train_doors,doors_sum,_0_9,_10_12,_13_15,_16_20,_21_25,_26_30,_31_60,_16plus,_60plus
count,7.000000,7.000000,7.000000,7.000000,7.000000,7.0,7.000000,7.000000,7.000000,7.000000,7.0,7.000000,7.000000,7.000000,7.000000,7.000000,7.0,7.000000,7.000000,7.000000,7.000000,7.000000,7.000000,7.000000,7.000000,7.000000,7.000000,7.000000,7.000000,7.000000,7.000000,7.000000,7.000000
mean,209.142857,150525.061328,125625.014430,118009.150823,32515.910505,0.0,161208.610881,128299.078963,136308.542777,32909.531918,0.0,281.000000,9.080841,152284.230107,12.428571,147.428571,0.0,14.142857,49.000000,24.857143,294.857143,49.000000,28.285714,397.000000,166.857143,45.285714,4.857143,58.857143,3.857143,0.571429,0.571429,63.857143,0.142857
std,81.795331,64356.670905,51698.982763,50353.193511,21829.293291,0.0,68063.875951,54263.336972,55622.662081,22291.801815,0.0,417.583125,5.002446,120751.172970,28.330532,203.846231,0.0,37.418483,49.662192,56.661064,407.692461,49.662192,74.836966,562.605546,234.832159,71.762837,9.063270,105.410987,10.205041,1.511858,1.511858,117.904522,0.377964
min,94.000000,81896.166667,63166.666667,54833.333333,0.000000,0.0,87475.207774,59130.136614,67962.595479,0.000000,0.0,6.000000,4.451499,0.000000,0.000000,0.000000,0.0,0.000000,6.000000,0.000000,0.000000,6.000000,0.000000,6.000000,6.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,152.000000,91972.777778,83711.416667,83711.416667,21200.000000,0.0,100232.458508,92362.638533,92362.638533,21200.018055,0.0,37.500000,7.050823,64374.404317,0.000000,22.000000,0.0,0.000000,11.000000,0.000000,44.000000,11.000000,0.000000,59.000000,20.000000,2.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,218.000000,159212.666667,131286.000000,109636.666667,29112.222222,0.0,170078.971124,119586.083225,142152.280674,29611.365451,0.0,78.000000,7.515625,168268.758600,0.000000,48.000000,0.0,0.000000,32.000000,0.000000,96.000000,32.000000,0.000000,120.000000,66.000000,5.000000,0.000000,8.000000,0.000000,0.000000,0.000000,8.000000,0.000000
75%,277.000000,192880.611111,160140.527778,154013.916667,47416.131212,0.0,205963.180008,166758.525380,173223.068792,47747.840477,0.0,347.000000,8.827882,220249.368443,5.500000,228.500000,0.0,0.000000,71.000000,11.000000,457.000000,71.000000,0.000000,539.000000,205.500000,68.000000,5.000000,60.500000,0.000000,0.000000,0.000000,60.500000,0.000000
max,294.000000,242859.818182,197218.545455,186143.389091,61266.888889,0.0,258514.820234,201135.005075,212873.508636,62859.640911,0.0,1114.000000,19.841354,328473.306633,76.000000,483.000000,0.0,99.000000,141.000000,152.000000,966.000000,141.000000,198.000000,1457.000000,645.000000,171.000000,24.000000,283.000000,27.000000,4.000000,4.000000,318.000000,1.000000


In [16]:
df_all.describe()

,grant_fiscal_year,project_year,allocationamount,encumbered_amount,expendedamount,activebalance,closedoutbalance,_0_9,_10_12,_13_15,_16_20,_21_25,_26_30,_31_60,_60plus,total_vehicles,average_age_of_fleet__in_years_,average_lifetime_miles_per_vehicle,sum_15plus,Automobiles,Bus,Other,Service,Train,Van,automobiles_door,bus_doors,train_doors,van_doors,doors_sum,itp_id,adjusted_allocationamount,adjusted_encumbered_amount,adjusted_expendedamount,adjusted_activebalance,adjusted_closedoutbalance,_16plus
count,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.0,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,6.0,49.000000,49.000000,49.0,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.0,49.000000
mean,2018.693878,2018.693878,161242.836735,132295.122449,122051.250612,39191.586122,0.0,148.000000,38.836735,3.530612,49.714286,3.306122,0.489796,0.489796,0.122449,244.489796,8.647712,126455.406408,319.0,10.653061,126.938776,0.0,49.510204,12.122449,45.265306,21.306122,253.877551,24.244898,45.265306,344.693878,214.448980,172040.893407,143093.154468,132316.149827,39724.743580,0.0,54.000000
std,1.758362,1.758362,79986.848659,99547.437204,98878.839594,72881.855248,0.0,209.129346,64.158121,7.929644,93.905981,8.942418,1.324803,1.324803,0.331201,373.017712,4.379657,105286.737085,0.0,24.925348,183.694766,0.0,103.652087,32.788865,44.296245,49.850695,367.389532,65.577731,44.296245,504.528460,72.075499,85761.333746,107261.285266,106940.436331,73772.476200,0.0,104.852078
min,2016.000000,2016.000000,35000.000000,0.000000,0.000000,0.000000,0.0,6.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.000000,4.451499,0.000000,319.0,0.000000,0.000000,0.0,0.000000,0.000000,6.000000,0.000000,0.000000,0.000000,6.000000,6.000000,94.000000,37768.583952,0.000000,0.000000,0.000000,0.0,0.000000
25%,2017.000000,2017.000000,83218.000000,50000.000000,0.000000,0.000000,0.0,8.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,10.000000,6.900000,0.000000,319.0,0.000000,0.000000,0.0,0.000000,0.000000,10.000000,0.000000,0.000000,0.000000,10.000000,10.000000,194.000000,86716.374547,52348.980323,0.000000,0.000000,0.0,0.000000
50%,2019.000000,2019.000000,158411.000000,148815.000000,135000.000000,0.000000,0.0,66.000000,3.000000,0.000000,8.000000,0.000000,0.000000,0.000000,0.000000,78.000000,7.515625,168268.758600,319.0,0.000000,44.000000,0.0,2.000000,0.000000,32.000000,0.000000,88.000000,0.000000,32.000000,120.000000,218.000000,168013.327535,157046.940968,143085.833625,0.000000,0.0,8.000000
75%,2020.000000,2020.000000,215087.000000,207226.000000,206437.000000,58802.000000,0.0,106.000000,10.000000,1.000000,19.000000,0.000000,0.000000,0.000000,0.000000,116.000000,7.812500,222854.095200,319.0,0.000000,56.000000,0.0,5.000000,0.000000,58.000000,0.000000,112.000000,0.000000,58.000000,170.000000,264.000000,228207.904954,227255.280333,225872.730213,58802.050078,0.0,19.000000
max,2021.000000,2021.000000,363122.000000,363122.000000,363122.000000,275001.000000,0.0,645.000000,171.000000,24.000000,283.000000,27.000000,4.000000,4.000000,1.000000,1114.000000,19.841354,328473.306633,319.0,76.000000,483.000000,0.0,315.000000,99.000000,141.000000,152.000000,966.000000,198.000000,141.000000,1457.000000,294.000000,391845.821200,391845.821200,391845.821200,275001.234202,0.0,318.000000


## Run Charting Functions 

In [17]:
def labeling(word):
    # Add specific use cases where it's not just first letter capitalized
    LABEL_DICT = { "prepared_y": "Year",
              "dist": "District",
              "nunique":"Number of Unique",
              "project_no": "Project Number"}
    
    if (word == "mpo") or (word == "rtpa"):
        word = word.upper()
    elif word in LABEL_DICT.keys():
        word = LABEL_DICT[word]
    else:
        #word = word.replace('n_', 'Number of ').title()
        word = word.replace('unique_', "Number of Unique ").title()
        word = word.replace('_', ' ').title()
    
    return word


def basic_bar_chart(df, x_col, y_col, color_col, chart_title=''):
    
    if chart_title == "":
        chart_title = (f"{labeling(x_col)} by {labeling(y_col)}")
    
    chart = (alt.Chart(df)
             .mark_bar()
             .encode(
                 x=alt.X(x_col, title=labeling(x_col), sort=('-y')),
                 y=alt.Y(y_col, title=labeling(y_col)),
                 #column = "payment:N",
                 color = alt.Color(color_col,
                                  scale=alt.Scale(
                                      range=altair_utils.CALITP_CATEGORY_BRIGHT_COLORS),
                                      legend=alt.Legend(title=(labeling(color_col)))
                                  ))
             .properties( 
                          title=chart_title)
    )

    chart=styleguide.preset_chart_config(chart)
    
    return chart

## Charts

In [18]:
basic_bar_chart(df2, 'organization_name', 'total_vehicles', 'organization_name', chart_title='Fleet Size by Agency')

alt.Chart(...)

In [19]:
basic_bar_chart(df2, 'organization_name', 'adjusted_allocationamount', 'organization_name', chart_title='Aveage Allocation Amount Size by Agency')

alt.Chart(...)

In [20]:
basic_bar_chart(df2, 'organization_name',
                'average_age_of_fleet__in_years_',
                'organization_name',
                chart_title='Average Fleet Age by Agency')

alt.Chart(...)

In [21]:
chart = (alt.Chart(df_all)
             .mark_bar(size=30)
             .encode(
                 x=alt.X('project_year:O', title=labeling('project_year')),
                 y=alt.Y('adjusted_allocationamount:Q', title=('Allocation Amount ($2021)')),
                 color = alt.Color('organization_name:N',
                                  scale=alt.Scale(
                                      range=altair_utils.CALITP_CATEGORY_BRIGHT_COLORS),
                                      legend=alt.Legend(title=(labeling('organization_name')))
                                  )
             )
             .properties( 
                          title=('Allocation Amount by Year'))
    )

chart=styleguide.preset_chart_config(chart)
    
display(chart)

alt.Chart(...)

In [27]:
df_count = df_all>>group_by(_.project_year)>>count(_.organization_name)>>arrange(-_.n)
df_count

,project_year,organization_name,n
23,2019,Napa Valley Transportation Authority,3
32,2021,City of Dixon,3
33,2021,City of Rio Vista,3
11,2017,Napa Valley Transportation Authority,2
17,2018,Napa Valley Transportation Authority,2
21,2019,City of Rio Vista,2
26,2020,City of Dixon,2
35,2021,Napa Valley Transportation Authority,2
0,2016,City of Dixon,1
1,2016,City of Fairfield,1


In [28]:
chart = (alt.Chart(df_count)
             .mark_bar(size=30)
             .encode(
                 x=alt.X('project_year:O', title=labeling('project_year')),
                 y=alt.Y('n:Q', title=('Number of Projects')),
                 color = alt.Color('organization_name:N',
                                  scale=alt.Scale(
                                      range=altair_utils.CALITP_CATEGORY_BRIGHT_COLORS),
                                      legend=alt.Legend(title=(labeling('organization_name')))
                                  )
    #             , column='organization_name:N'
             )
             .properties( 
                          title=('Number of Projects by Year'))
    )

chart=styleguide.preset_chart_config(chart)
    
display(chart)

alt.Chart(...)